# Week 3: Mathematical Foundations for Machine Learning

From PCA to Gradient Descent with real-world examples from Amazon, Netflix, and Google.

## Topics Covered
1. Linear Algebra: Eigendecomposition, SVD
2. PCA: Theory and Implementation
3. Gradient Descent: Optimization
4. Naive Bayes: Probabilistic Classification
5. EM Algorithm: Gaussian Mixture Models
6. SVM: Kernel Trick

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

## 1. Eigendecomposition

For a square matrix A, eigenvector **v** and eigenvalue **λ** satisfy:

$$Av = \lambda v$$

**Intuition**: Eigenvectors are directions that only get scaled, not rotated.

In [ ]:
# Eigendecomposition from scratch
def power_iteration(A, num_iters=100):
    """Find dominant eigenvector using power iteration."""
    n = A.shape[0]
    v = np.random.randn(n)
    v = v / np.linalg.norm(v)
    
    for _ in range(num_iters):
        Av = A @ v
        v = Av / np.linalg.norm(Av)
    
    eigenvalue = (v @ A @ v) / (v @ v)
    return eigenvalue, v

A = np.array([[2., 1.], [1., 2.]])
val, vec = power_iteration(A)
print(f'Dominant eigenvalue: {val:.4f}')
print(f'Eigenvector: {vec}')

## 2. Singular Value Decomposition (SVD)

Any matrix A can be decomposed:

$$A = U \Sigma V^T$$

**Netflix Application**: Decompose user-movie ratings into latent factors.

In [ ]:
# SVD for Movie Recommendations (Netflix-style)
# Ratings matrix: users x movies (? = missing)
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4]
], dtype=float)

# Fill missing with mean for SVD
R_filled = R.copy()
R_filled[R_filled == 0] = R[R != 0].mean()

# SVD decomposition
U, S, Vt = np.linalg.svd(R_filled, full_matrices=False)

# Low-rank approximation (k=2 latent factors)
k = 2
R_approx = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]

print('Original ratings (0=missing):')
print(R.astype(int))
print('\nPredicted ratings:')
print(np.round(R_approx, 1))

## 3. Principal Component Analysis (PCA)

**Goal**: Find orthogonal directions of maximum variance.

**Steps**:
1. Center data: $X_c = X - \mu$
2. Covariance: $C = \frac{1}{n} X_c^T X_c$
3. Eigendecompose C → principal components

In [ ]:
class PCAFromScratch:
    """PCA implementation from scratch."""
    
    def __init__(self, n_components):
        self.n_components = n_components
        self.components_ = None
        self.explained_variance_ = None
    
    def fit(self, X):
        # Center data
        self.mean_ = X.mean(axis=0)
        X_centered = X - self.mean_
        
        # Covariance matrix
        cov = X_centered.T @ X_centered / (len(X) - 1)
        
        # Eigendecomposition
        eigenvalues, eigenvectors = np.linalg.eigh(cov)
        
        # Sort by eigenvalue (descending)
        idx = eigenvalues.argsort()[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        self.components_ = eigenvectors[:, :self.n_components].T
        self.explained_variance_ = eigenvalues[:self.n_components]
        return self
    
    def transform(self, X):
        X_centered = X - self.mean_
        return X_centered @ self.components_.T

# Example: Reduce 4D to 2D
X = np.random.randn(100, 4)
pca = PCAFromScratch(n_components=2).fit(X)
X_reduced = pca.transform(X)
print(f'Original shape: {X.shape} → Reduced: {X_reduced.shape}')
print(f'Explained variance: {pca.explained_variance_}')

## 4. Gradient Descent

**Update Rule**:
$$\theta_{new} = \theta_{old} - \alpha \nabla L(\theta)$$

**Intuition**: Walk downhill on the loss surface.

In [ ]:
def gradient_descent(f, grad_f, x0, lr=0.1, n_iters=50):
    """General gradient descent.
    
    Args:
        f: Function to minimize
        grad_f: Gradient of f
        x0: Starting point
        lr: Learning rate (α)
        n_iters: Number of iterations
    """
    x = x0.copy()
    history = [x.copy()]
    
    for _ in range(n_iters):
        grad = grad_f(x)
        x = x - lr * grad  # THE UPDATE RULE
        history.append(x.copy())
    
    return x, history

# Example: Minimize f(x,y) = x² + y²
f = lambda x: x[0]**2 + x[1]**2
grad_f = lambda x: np.array([2*x[0], 2*x[1]])

x_min, hist = gradient_descent(f, grad_f, np.array([3., 4.]))
print(f'Minimum found at: {x_min}')
print(f'f(minimum) = {f(x_min):.6f}')

## 5. Naive Bayes Classifier (Google Spam Filter)

**Bayes' Theorem**:
$$P(spam|words) = \frac{P(words|spam) \cdot P(spam)}{P(words)}$$

**Naive Assumption**: Words are independent given class.

In [ ]:
class NaiveBayesSpam:
    """Naive Bayes spam classifier (simplified)."""
    
    def __init__(self, alpha=1.0):
        self.alpha = alpha  # Laplace smoothing
        self.word_probs = {'spam': {}, 'ham': {}}
        self.class_probs = {}
    
    def fit(self, emails, labels):
        spam_count = sum(labels)
        ham_count = len(labels) - spam_count
        
        self.class_probs = {
            'spam': spam_count / len(labels),
            'ham': ham_count / len(labels)
        }
        
        # Count words
        vocab = set()
        for email in emails:
            vocab.update(email.lower().split())
        
        spam_words = []
        ham_words = []
        for email, label in zip(emails, labels):
            words = email.lower().split()
            if label:
                spam_words.extend(words)
            else:
                ham_words.extend(words)
        
        # Laplace-smoothed probabilities
        for word in vocab:
            self.word_probs['spam'][word] = (
                spam_words.count(word) + self.alpha
            ) / (len(spam_words) + self.alpha * len(vocab))
            self.word_probs['ham'][word] = (
                ham_words.count(word) + self.alpha
            ) / (len(ham_words) + self.alpha * len(vocab))
    
    def predict(self, email):
        words = email.lower().split()
        log_spam = np.log(self.class_probs['spam'])
        log_ham = np.log(self.class_probs['ham'])
        
        for word in words:
            if word in self.word_probs['spam']:
                log_spam += np.log(self.word_probs['spam'][word])
                log_ham += np.log(self.word_probs['ham'][word])
        
        return 'spam' if log_spam > log_ham else 'ham'

# Training data
emails = [
    'free money click now', 'winner lottery claim prize',
    'meeting tomorrow morning', 'project report attached',
    'urgent claim your free gift', 'lunch plans today'
]
labels = [1, 1, 0, 0, 1, 0]  # 1=spam, 0=ham

clf = NaiveBayesSpam()
clf.fit(emails, labels)

print(clf.predict('free prize winner'))       # spam
print(clf.predict('meeting scheduled today'))  # ham

## 6. EM Algorithm for GMMs

**E-Step**: Compute responsibilities (soft assignments)
$$r_{nk} = \frac{\pi_k \mathcal{N}(x_n|\mu_k, \Sigma_k)}{\sum_j \pi_j \mathcal{N}(x_n|\mu_j, \Sigma_j)}$$

**M-Step**: Update parameters using responsibilities

In [ ]:
def gaussian_pdf(x, mu, sigma):
    """Univariate Gaussian PDF."""
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2) / (sigma * np.sqrt(2 * np.pi))

def em_gmm_1d(X, K=2, n_iters=20):
    """EM for 1D Gaussian Mixture Model."""
    n = len(X)
    
    # Initialize
    mus = np.random.choice(X, K)
    sigmas = np.ones(K)
    pis = np.ones(K) / K
    
    for _ in range(n_iters):
        # E-STEP: Compute responsibilities
        resp = np.zeros((n, K))
        for k in range(K):
            resp[:, k] = pis[k] * gaussian_pdf(X, mus[k], sigmas[k])
        resp /= resp.sum(axis=1, keepdims=True)
        
        # M-STEP: Update parameters
        Nk = resp.sum(axis=0)
        for k in range(K):
            mus[k] = (resp[:, k] @ X) / Nk[k]
            sigmas[k] = np.sqrt((resp[:, k] @ (X - mus[k])**2) / Nk[k])
            pis[k] = Nk[k] / n
    
    return mus, sigmas, pis

# Generate bimodal data
X = np.concatenate([np.random.normal(-2, 0.5, 50),
                    np.random.normal(2, 0.8, 50)])

mus, sigmas, pis = em_gmm_1d(X, K=2)
print(f'Found means: {mus}')
print(f'Found stds: {sigmas}')
print(f'Mixture weights: {pis}')

## 7. SVM with Kernel Trick

**Kernel**: Implicit inner product in high-dimensional space

$$k(x, z) = \phi(x)^T \phi(z)$$

**RBF Kernel**: $k(x, z) = \exp(-\gamma ||x-z||^2)$

In [ ]:
def rbf_kernel(X1, X2, gamma=1.0):
    """RBF (Gaussian) kernel.
    
    The kernel trick allows linear algorithms to
    work in infinite-dimensional feature spaces!
    """
    # Compute pairwise squared distances
    sq_dist = np.sum(X1**2, axis=1).reshape(-1, 1) + \
              np.sum(X2**2, axis=1) - 2 * X1 @ X2.T
    return np.exp(-gamma * sq_dist)

# Example: XOR problem (not linearly separable)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])  # XOR

# Kernel matrix
K = rbf_kernel(X, X, gamma=1.0)
print('RBF Kernel Matrix (allows non-linear separation):')
print(np.round(K, 3))

## Summary: Real-World Applications

| Algorithm | Company | Application |
|-----------|---------|-------------|
| SVD | Netflix | Movie recommendations |
| PCA | Amazon | Feature reduction for forecasting |
| Naive Bayes | Google | Spam filtering |
| Gradient Descent | All | Training neural networks |
| EM | Many | Customer segmentation |
| SVM | Various | Text classification |